# Deep Learning to Classify images


Deep learning (also known as deep structured learning) is a machine learning system that uses artificial neural networks to learn representations. Supervised, semi-supervised, and unsupervised learning are all possible options.

Here in This project, Every picture has a cell nucleus in the centre. The ultimate aim for this data science project is to train a deep neural network that can classify a 64x64 image with a cell nucleus in the centre into one of the following forms:

1. Normal epithelial cells.
2. Cancer epithelial cells.
3. Immune Leukocyte cells .
4. Connective fibroblast cells .

### Importing necessary libraries

In [21]:
import os
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torchvision.io import read_image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
#file path
path = 'deep-learning-for-msc-coursework-2021/'

# creating dataframe

train_df = pd.read_csv(path + 'train.csv')
train_df['Image'] = train_df['Id'].astype(str) + '.png'
train_df = train_df[train_df.columns[[0, 2, 1]]]
train_df['Type'] = train_df['Type'].astype('category')
train_df['Labels'] = train_df['Type'].cat.codes

train_df.head

<bound method NDFrame.head of         Id     Image         Type  Labels
0        1     1.png   Connective       1
1        2     2.png   Connective       1
2        3     3.png   Connective       1
3        4     4.png   Connective       1
4        5     5.png   Connective       1
...    ...       ...          ...     ...
2185  2186  2186.png       Immune       2
2186  2187  2187.png       Cancer       0
2187  2188  2188.png   Connective       1
2188  2189  2189.png   Connective       1
2189  2190  2190.png       Immune       2

[2190 rows x 4 columns]>

###  Creating Class Structure to apply transformation to training set


In [23]:
class CIDataset(Dataset):
    def __init__(self, data_frame, image_dir, transform=None):
        self.data_frame = data_frame
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_name  = os.path.join(self.image_dir, self.data_frame.iloc[idx, 1])
        images = read_image(image_name)
        labels = torch.from_numpy(np.array(self.data_frame.iloc[idx, -1])).long()
        images = images.type(torch.FloatTensor)
        images = images / 255.0
        if self.transform:
            images = self.transform(images)
            
        return (images, labels)

## Data Augumentation

Data augmentation, which involves randomly transforming the training images, such as flipping, rotating, or cropping them, is one way to mitigate the overfitting problem.

Since these data augmentation transformations are implemented at random during training, the same image can be interpreted differently from batch to batch, resulting in more variety in the training data and assisting in the development of new tools.

In [24]:
train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [25]:
train_dataset = CIDataset(
    data_frame = train_df,
    image_dir = path + 'train/train',
    transform = train_transform)


## Modeling Neural Network
A CNN's central building block is the convolutional layer. The parameters of the layer are made up of a series of learnable filters (or kernels) with a limited receptive field but that span the entire depth of the input volume.

Each filter is convolved across the width and height of the input volume during the forward transfer, computing the dot product between the filter's entries and the input, and generating a 2-dimensional activation map for that filter. As a consequence, the network learns filters that activate when it detects a certain form of feature at a particular spatial location in the input.


We have three convolution layers and three linear layers in this model. Each layer has a Relu activation function added to it to make it non-linear.

In [51]:
class cnNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3,kernel_size=3,out_channels=16,stride=1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=16,kernel_size=3,out_channels=64,stride=1,padding=1)
        self.conv3 = nn.Conv2d(in_channels=64,kernel_size=3,out_channels=128,stride=1,padding=1)
        #self.conv4 = nn.Conv2d(in_channels=128,kernel_size=3,out_channels=256,stride=1,padding=1)
        self.fc1 = nn.Linear(8*8*128, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 4)
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, 0.3)
        
        x = F.relu(self.conv3(x))
        x = F.avg_pool2d(x, 2, 2)
        
        x = x.view(-1, 8*8*128)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

### Loading train data using DataLoader

In [52]:
torch.manual_seed(50)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

### Checking for Availability of GPU

In [53]:
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
cuda_avail = torch.cuda.is_available()
print (cuda_avail)
print (device)

True
cuda


### Creating Model Object

In [54]:
torch.manual_seed(101)
model = cnNet()
if cuda_avail:
    model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
model

cnNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=4, bias=True)
)

### Adjusting Learning rate

In [55]:
#adjusting the learning rate based on number of epochs

def adjust_learning_rate(epoch):
    lr = 0.001
    if epoch > 55:
        lr = lr/10000
    if epoch > 50:
        lr = lr / 10000
    elif epoch > 45:
        lr = lr / 1000
    elif epoch > 35:
        lr = lr / 100
    elif epoch > 20:
        lr = lr / 10
    
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

### Training the Model

In [56]:
epochs = 60
n_epoch = 5

train_losses = []
train_correct = []

best_acc = 0

for i in range(1, epochs +1):
    train_acc = 0
    
    for b, (X_train, y_train) in enumerate(train_loader):
       
        y_pred = model(X_train.to(device))
        loss = loss_fn(y_pred, y_train.to(device))
        predicted = torch.max(y_pred.data, 1)[1]
        train_acc += (predicted == y_train.to(device)).sum()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    adjust_learning_rate(i)
        
    train_losses.append(loss)
    
    acc = train_acc.item()*100/2190
    train_correct.append(acc)
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), 'best-model.pt')
    
    if i ==1 or i % n_epoch == 0:
        print(f'Epoch:{i} \t Loss:{loss.item():10.8f} \t Train accuracy:{acc:7.3f}%')

Epoch:1 	 Loss:0.62451327 	 Train accuracy: 48.767%
Epoch:5 	 Loss:0.66178089 	 Train accuracy: 72.100%
Epoch:10 	 Loss:0.58386821 	 Train accuracy: 77.671%
Epoch:15 	 Loss:0.45930877 	 Train accuracy: 80.502%
Epoch:20 	 Loss:0.46861848 	 Train accuracy: 82.283%
Epoch:25 	 Loss:0.54915631 	 Train accuracy: 86.941%
Epoch:30 	 Loss:0.22757420 	 Train accuracy: 88.767%
Epoch:35 	 Loss:0.11863355 	 Train accuracy: 90.183%
Epoch:40 	 Loss:0.17650083 	 Train accuracy: 90.959%
Epoch:45 	 Loss:0.34122354 	 Train accuracy: 90.776%
Epoch:50 	 Loss:0.20314768 	 Train accuracy: 91.735%
Epoch:55 	 Loss:0.10339920 	 Train accuracy: 91.005%
Epoch:60 	 Loss:0.50156283 	 Train accuracy: 91.279%


### Loading the Test data

In [57]:
root = 'deep-learning-for-msc-coursework-2021/'
valid_dataset = datasets.ImageFolder(os.path.join(root, 'test'), transform=test_transform)

torch.manual_seed(42)
valid_loader = DataLoader(valid_dataset, batch_size=10, shuffle=False)

### Predicting the Test Data

In [58]:
diseases = ['Cancer', 'Connective', 'Immune', 'Normal']

predictions = []

# loading the best model parameters
model.load_state_dict(torch.load('best-model.pt'))
model.eval()

with torch.no_grad():
    for images,labels in valid_loader:
        out = model(images.to(device))
        predicted = torch.max(out.data, 1)[1]
        predictions.append(predicted)
predictions = torch.cat(predictions)
id = np.arange(10001, 10401)
type = np.array([diseases[i] for i in predictions])
dict = {'Id': id, 'Type': type}
df = pd.DataFrame(dict)
df.to_csv('test.csv', index=False)

print(df)

        Id        Type
0    10001  Connective
1    10002  Connective
2    10003  Connective
3    10004      Immune
4    10005  Connective
..     ...         ...
395  10396      Cancer
396  10397      Cancer
397  10398      Cancer
398  10399      Cancer
399  10400      Normal

[400 rows x 2 columns]


In [25]:
#plt.plot(train_correct)